In [824]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter
import nltk

# nltk.download('gutenberg')
#!python -m spacy download en

Supervised NLP requires a pre-labelled dataset for training and testing, and is generally interested in categorizing text in various ways. In this case, we are going to try to predict whether a sentence comes from _Alice in Wonderland_ by Lewis Carroll or _Persuasion_ by Jane Austen. We can use any of the supervised models we've covered previously, as long as they allow categorical outcomes. In this case, we'll try Random Forests, SVM, and KNN.

Our feature-generation approach will be something called _BoW_, or _Bag of Words_. BoW is quite simple: For each sentence, we count how many times each word appears. We will then use those counts as features.

**Note**: Since processing all the text takes around ~5-10 minutes, in the cell below we are taking only the first tenth of each text. If you want to experiment, feel free to change the following code in the next cell:

```python
alice = text_cleaner(alice[:int(len(alice)/10)])
persuasion = text_cleaner(persuasion[:int(len(persuasion)/10)])
```
to 

```python
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)
```

In [825]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice[:int(len(alice)/10)])
persuasion = text_cleaner(persuasion[:int(len(persuasion)/10)])

In [826]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [827]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.tail()

,0,1
470,"(Had, she, not, imagined, herself, consulting,...",Austen
471,"(The, belief, of, being, prudent, ,, and, self...",Austen
472,"(ill, used, by, so, forced, a, relinquishment, .)",Austen
473,"(He, had, left, the, country, in, consequence, .)",Austen
474,"(A, few, months, had, seen, the, beginning, an...",Austen


Time to bag some words!  Since spaCy has already tokenized and labelled our data, we can move directly to recording how often various words occur.  We will exclude stopwords and punctuation.  In addition, in an attempt to keep our feature space from exploding, we will work with lemmas (root words) rather than the raw text terms, and we'll only use the 2000 most common words for each text.

In [828]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 50 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [829]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250
Processing row 300
Processing row 350
Processing row 400
Processing row 450


,minute,ask,patent,rule,plenty,authority,confess,late,infected,rate,...,idle,powerless,Basil,answer,glad,suspicious,custom,curtsey,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


## Trying out BoW

Now let's give the bag of words features a whirl by trying a random forest.

In [830]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.9614035087719298

Test set score: 0.7842105263157895


/home/uzi/.venv/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Holy overfitting, Batman! Overfitting is a known problem when using bag of words, since it basically involves throwing a massive number of features at a model – some of those features (in this case, word frequencies) will capture noise in the training set. Since overfitting is also a known problem with Random Forests, the divergence between training score and test score is expected.


## BoW with Logistic Regression

Let's try a technique with some protection against overfitting due to extraneous features – logistic regression with ridge regularization (from ridge regression, also called L2 regularization).

In [831]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(penalty='l2') # No need to specify l2 as it's the default. But we put it for demonstration.
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(285, 1608) (285,)
Training set score: 0.968421052631579

Test set score: 0.8631578947368421


/home/uzi/.venv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [832]:
pd.crosstab(y_test, lr.predict(X_test))

col_0,Austen,Carroll
text_source,,
Austen,106,17
Carroll,9,58


In [833]:
from sklearn.model_selection import cross_val_score

print(cross_val_score(lr, X, Y, cv=5))

/home/uzi/.venv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/uzi/.venv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/uzi/.venv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/uzi/.venv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


[0.84375    0.82291667 0.71578947 0.85106383 0.79787234]


/home/uzi/.venv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Logistic regression performs a bit better than the random forest.  

# BoW with Gradient Boosting

And finally, let's see what gradient boosting can do:

In [834]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.8842105263157894

Test set score: 0.7894736842105263


Looks like logistic regression is the winner, but there's room for improvement.

# Same model, new inputs

What if we feed the model a different novel by Jane Austen, like *Emma*?  Will it be able to distinguish Austen from Carroll with the same level of accuracy if we insert a different sample of Austen's writing?

First, we need to process _Emma_ the same way we processed the other data, and combine it with the Alice data. Remember that for computation time concerns, we only took the first tenth of the Alice text. Emma is pretty long. **So in order to get comparable length texts, we take the first sixtieth of Emma**. Again, if you want to experiment, you can take the whole texts of each.

In [835]:
# Clean the Emma data.
emma = gutenberg.raw('austen-emma.txt')
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma[:int(len(emma)/60)])
print(emma[:100])

Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to


In [836]:
# Parse our cleaned data.
emma_doc = nlp(emma)

In [837]:
# Group into sentences.
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]

In [838]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
emma_sentences = pd.DataFrame(emma_sents)
emma_bow = bow_features(emma_sentences, common_words)

print('done')

Processing row 0
Processing row 50
Processing row 100
done


In [839]:
# Now we can model it!
# Let's use logistic regression again.

# Combine the Emma sentence data with the Alice data from the test set.
X_Emma_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    emma_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * emma_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted, rownames=['Actual v'], colnames=['Predicted >'])


Test set score: 0.452


Predicted >,Austen,Carroll
Actual v,,
Austen,72,78
Carroll,59,41


Well look at that!  NLP approaches are generally effective on the same type of material as they were trained on. It looks like this model is actually able to differentiate multiple works by Austen from Alice in Wonderland.  Now the question is whether the model is very good at identifying Austen, or very good at identifying Alice in Wonderland, or both...

# Challenge 0:

Recall that the logistic regression model's best performance on the test set was 93%.  See what you can do to improve performance.  Suggested avenues of investigation include: Other modeling techniques (SVM?), making more features that take advantage of the spaCy information (include grammar, phrases, POS, etc), making sentence-level features (number of words, amount of punctuation), or including contextual information (length of previous and next sentences, words repeated from one sentence to the next, etc), and anything else your heart desires.  Make sure to design your models on the test set, or use cross_validation with multiple folds, and see if you can get accuracy above 90%.  

# Challenge 1:
Find out whether your new model is good at identifying Alice in Wonderland vs any other work, Persuasion vs any other work, or Austen vs any other work.  This will involve pulling a new book from the Project Gutenberg corpus (print(gutenberg.fileids()) for a list) and processing it.

Record your work for each challenge in a notebook and submit it below.

# Challenge 0

## Support Vector Machines

In [840]:
y_train.value_counts()

Austen     185
Carroll    100
Name: text_source, dtype: int64

In [841]:
from sklearn.svm import SVC

svc = SVC(class_weight='balanced')
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
svc.fit(X_train, y_train)

print('Training set score:', svc.score(X_train, y_train))
print('\nTest set score:', svc.score(X_test, y_test))

/home/uzi/.venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Training set score: 0.3508771929824561

Test set score: 0.3526315789473684


In [842]:
# Model.
print('\nTest set score:', svc.score(X_test, y_test))
svc_predicted = svc.predict(X_test)
pd.crosstab(y_test, svc_predicted, rownames=['Actual v'], colnames=['Predicted >'])


Test set score: 0.3526315789473684


Predicted >,Carroll
Actual v,
Austen,123
Carroll,67


In [843]:
svc = SVC()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

svc.fit(X_train, y_train)

print('Training set score:', svc.score(X_train, y_train))
print('\nTest set score:', svc.score(X_test, y_test))

/home/uzi/.venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Training set score: 0.6491228070175439

Test set score: 0.6473684210526316


In [844]:
print('\nTest set score:', svc.score(X_test, y_test))
svc_predicted = svc.predict(X_test)
pd.crosstab(y_test, svc_predicted, rownames=['Actual v'], colnames=['Predicted >'])


Test set score: 0.6473684210526316


Predicted >,Austen
Actual v,
Austen,123
Carroll,67


# KNN

In [845]:
from sklearn.neighbors import KNeighborsClassifier
for n in range(1,6):
    knn = KNeighborsClassifier(n_neighbors=n)
    
    knn.fit(X_train, y_train)
    
    print(f'{n}-Neighbor(s):')
    print('\nTraining set score:', knn.score(X_train, y_train))
    print('Test set score:', knn.score(X_test, y_test))
    print('-'*50)
    

1-Neighbor(s):

Training set score: 0.968421052631579
Test set score: 0.6368421052631579
--------------------------------------------------
2-Neighbor(s):

Training set score: 0.9122807017543859
Test set score: 0.6368421052631579
--------------------------------------------------
3-Neighbor(s):

Training set score: 0.6982456140350877
Test set score: 0.5526315789473685
--------------------------------------------------
4-Neighbor(s):

Training set score: 0.7684210526315789
Test set score: 0.5947368421052631
--------------------------------------------------
5-Neighbor(s):

Training set score: 0.6140350877192983
Test set score: 0.5052631578947369
--------------------------------------------------


In [846]:
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)

print('\nTest set score:', knn.score(X_test, y_test))
knn_predicted = knn.predict(X_test)
pd.crosstab(y_test, knn_predicted, rownames=['Actual v'], colnames=['Predicted >'])


Test set score: 0.6368421052631579


Predicted >,Austen,Carroll
Actual v,,
Austen,66,57
Carroll,12,55


> None of these other algorithms performed any better than the logarithmic regression model. Next, we'll try Naive Bayes, using BoW to create our keyword list. So far, our logarithmic regression model has scored the following:

Training set score: 0.968421052631579

Test set score: 0.8631578947368421

> With the following cross validation scores:

[0.84375, 0.82291667, 0.71578947, 0.85106383, 0.79787234]

# BoW with Naive Bayes

In [847]:
# Set up the bags.
persuasionwords = bag_of_words(persuasion_doc)

# Create a set of Austen specific and common words
austen_words = set(persuasionwords)

In [848]:
# let's turn the 'text_source' column into a boolean so we can easily do some statistics to prepare for modeling.
word_counts['is_Austen'] = (word_counts['text_source'] == 'Austen')

In [849]:
Y = word_counts['is_Austen']
X = word_counts[austen_words]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=0)

In [850]:
# Our data is binary / boolean, so we're importing the Bernoulli classifier.
from sklearn.naive_bayes import BernoulliNB

# Instantiate our model and store it in a new variable.
bnb = BernoulliNB()

# Fit our model to the data.
bnb.fit(X_train,y_train)

# Classify, storing the result in a new variable.
y_pred = bnb.predict(X_test)

# Display our results.
print("Number of mislabeled points out of a total {} points : {}".format(
    X.shape[0],
    (y_test != y_pred).sum()
))

Number of mislabeled points out of a total 475 points : 56


In [851]:
bnb.score(X_test, y_test)

0.7052631578947368

In [852]:
pd.crosstab(y_test, y_pred)

col_0,False,True
is_Austen,,
False,65,2
True,54,69


# Feature Engineering

In [853]:
# Receate our original dataframe. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250
Processing row 300
Processing row 350
Processing row 400
Processing row 450


,minute,ask,patent,rule,plenty,authority,confess,late,infected,rate,...,idle,powerless,Basil,answer,glad,suspicious,custom,curtsey,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


In [854]:
# Creating a average word length per sentence feature (Austen seems to use bigger words more often than Carroll)
word_counts['avg_wordlength'] = 0.0

for idx, sent in  enumerate(word_counts['text_sentence']):
    nums = []
    for word in sent:
        if not word.is_stop and not word.is_punct:
            nums.append(len(word))
    word_counts['avg_wordlength'][idx] = np.mean(nums)

/home/uzi/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/home/uzi/.venv/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/uzi/.venv/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [855]:
word_counts.head(5)

,minute,ask,patent,rule,plenty,authority,confess,late,infected,rate,...,powerless,Basil,answer,glad,suspicious,custom,curtsey,text_sentence,text_source,avg_wordlength
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll,6.500000
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll,5.608696
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll,4.875000
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll,3.000000
4,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll,4.500000


In [856]:
word_counts[word_counts['avg_wordlength'].isna()].head()

,minute,ask,patent,rule,plenty,authority,confess,late,infected,rate,...,powerless,Basil,answer,glad,suspicious,custom,curtsey,text_sentence,text_source,avg_wordlength
17,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(Down, ,, down, ,, down, .)",Carroll,NaN
35,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,('),Carroll,NaN
38,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(Down, ,, down, ,, down, .)",Carroll,NaN
73,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,(*),Carroll,NaN
74,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,(*),Carroll,NaN


In [857]:
# Drop NaN values then reset index
word_counts.dropna(subset=['avg_wordlength'], inplace=True)
word_counts.reset_index(drop=True)

,minute,ask,patent,rule,plenty,authority,confess,late,infected,rate,...,powerless,Basil,answer,glad,suspicious,custom,curtsey,text_sentence,text_source,avg_wordlength
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll,6.500000
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll,5.608696
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll,4.875000
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll,3.000000
4,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll,4.500000
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"((, when, she, thought, it, over, afterwards, ...",Carroll,6.269231
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(,, she, ran, across, the, field, after, it, ,...",Carroll,5.111111
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(In, another, moment, down, went, Alice, after...",Carroll,6.200000
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(The, rabbit, -, hole, went, straight, on, lik...",Carroll,5.705882
9,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,"(Either, the, well, was, very, deep, ,, or, sh...",Carroll,4.900000


In [858]:
Y = word_counts['text_source']

X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=0.4,random_state=0)

In [859]:
lr.fit(X_train,y_train)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

Training set score: 0.968503937007874

Test set score: 0.8823529411764706


/home/uzi/.venv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


> An improvement over the previous lr scores of:

Training set score: 0.968421052631579

Test set score: 0.8631578947368421

In [860]:
pd.crosstab(y_test, lr.predict(X_test))

col_0,Austen,Carroll
text_source,,
Austen,117,2
Carroll,18,33


> Adding the avg_wordlength feature definitely improved detection (true positives) of Austen's work (from 106 to 117) but worsened detection of Carroll's work (from 58 to 33). Overall, accuracy imcreased.

In [861]:
from sklearn.model_selection import cross_val_score

print(cross_val_score(lr, X, Y, cv=5))

/home/uzi/.venv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/uzi/.venv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/uzi/.venv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/uzi/.venv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


[0.91764706 0.87058824 0.94117647 0.83529412 0.86904762]


/home/uzi/.venv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


> compare thes to the previous cross validation scores:

[0.84375, 0.82291667, 0.71578947, 0.85106383, 0.79787234]

# Challenge 1:

In [862]:
# Grab and process the raw data.
print(gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [863]:
# Load and clean the data.
bible = gutenberg.raw('bible-kjv.txt')

# The Chapter indicator is idiosyncratic
# persuasion = re.sub(r'Chapter \d+', '', persuasion)
bible = re.sub(r'\d:\d*', '', bible)
    
# Clean the text, bible has a lot more sentences than persuasion
bible = text_cleaner(bible[:int(len(bible)/120)])

In [864]:
bible_doc = nlp(bible)
bible[:100]

'The Old Testament of the King James Bible The First Book of Moses: Called Genesis In the beginning G'

In [865]:
# Group into sentences.
bible_sents = [[sent, "kjv"] for sent in bible_doc.sents]

# Combine the sentences from the bible and persuasion into one data frame.
sentences = pd.DataFrame(bible_sents + persuasion_sents)
sentences.head()

,0,1
0,"(The, Old, Testament, of, the, King, James, Bi...",kjv
1,"(The, First, Book, of, Moses, :, Called, Genesis)",kjv
2,"(In, the, beginning, God, created, the, heaven...",kjv
3,"(And, the, earth, was, without, form, ,, and, ...",kjv
4,"(And, the, Spirit, of, God, moved, upon, the, ...",kjv


In [918]:
# Set up the bags.
biblewords = bag_of_words(bible_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(biblewords + persuasionwords)

In [920]:
# Create our data frame with features. This can take a while to run.
word_counts_2 = bow_features(sentences, common_words)
word_counts_2.head()

Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250
Processing row 300
Processing row 350
Processing row 400
Processing row 450
Processing row 500
Processing row 550
Processing row 600
Processing row 650


,enter,ask,patent,Caphtorim,rule,hill,Genesis,authority,doth,confess,...,Basil,Ur,hundredth,answer,glad,Testament,suspicious,custom,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,"(The, Old, Testament, of, the, King, James, Bi...",kjv
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,"(The, First, Book, of, Moses, :, Called, Genesis)",kjv
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(In, the, beginning, God, created, the, heaven...",kjv
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(And, the, earth, was, without, form, ,, and, ...",kjv
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(And, the, Spirit, of, God, moved, upon, the, ...",kjv


In [922]:
# Creating a average word length per sentence feature (Austen seems to use bigger words more often than Carroll)
word_counts_2['avg_wordlength'] = 0.0

for idx, sent in  enumerate(word_counts_2['text_sentence']):
    nums = []
    for word in sent:
        if not word.is_stop and not word.is_punct:
            nums.append(len(word))
    word_counts_2['avg_wordlength'][idx] = np.mean(nums)

/home/uzi/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/home/uzi/.venv/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/uzi/.venv/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [923]:
word_counts_2[word_counts_2.text_source == 'Austen'].head(100)

,enter,ask,patent,Caphtorim,rule,hill,Genesis,authority,doth,confess,...,Ur,hundredth,answer,glad,Testament,suspicious,custom,text_sentence,text_source,avg_wordlength
358,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(Sir, Walter, Elliot, ,, of, Kellynch, Hall, ,...",Austen,7.200000
359,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(This, was, the, page, at, which, the, favouri...",Austen,6.142857
360,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(Walter, Elliot, ,, born, March, 1, ,, 1760, ,...",Austen,5.133333
361,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(of, South, Park, ,, in, the, county, of, Glou...",Austen,5.285714
362,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(he, has, issue, Elizabeth, ,, born, June, 1, ...",Austen,4.190476
363,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(Precisely, such, had, the, paragraph, origina...",Austen,6.142857
364,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(Then, followed, the, history, and, rise, of, ...",Austen,6.796610
365,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"("", Vanity, was, the, beginning, and, the, end...",Austen,6.300000
366,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(He, had, been, remarkably, handsome, in, his,...",Austen,6.000000
367,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(Few, women, could, think, more, of, their, pe...",Austen,5.909091


In [924]:
word_counts_2[word_counts_2['avg_wordlength'].isna()].head()

,enter,ask,patent,Caphtorim,rule,hill,Genesis,authority,doth,confess,...,Ur,hundredth,answer,glad,Testament,suspicious,custom,text_sentence,text_source,avg_wordlength
456,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"("")",Austen,NaN
464,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"("")",Austen,NaN
465,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(What, !)",Austen,NaN
547,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"("", Indeed, !)",Austen,NaN
549,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"("")",Austen,NaN


In [925]:
# Drop NaN values then reset index
word_counts_2.dropna(subset=['avg_wordlength'], inplace=True)
word_counts_2.reset_index(drop=True)

,enter,ask,patent,Caphtorim,rule,hill,Genesis,authority,doth,confess,...,Ur,hundredth,answer,glad,Testament,suspicious,custom,text_sentence,text_source,avg_wordlength
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,"(The, Old, Testament, of, the, King, James, Bi...",kjv,5.200000
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,"(The, First, Book, of, Moses, :, Called, Genesis)",kjv,5.500000
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(In, the, beginning, God, created, the, heaven...",kjv,6.000000
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(And, the, earth, was, without, form, ,, and, ...",kjv,4.833333
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(And, the, Spirit, of, God, moved, upon, the, ...",kjv,4.800000
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(And, God, said, ,, Let, there, be, light, :, ...",kjv,4.000000
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(And, God, saw, the, light, ,, that, it, was, ...",kjv,4.750000
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(And, God, called, the, light, Day, ,, and, th...",kjv,5.142857
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(And, the, evening, and, the, morning, were, t...",kjv,5.666667
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(And, God, said, ,, Let, there, be, a, firmame...",kjv,5.100000


In [926]:
word_counts_2[word_counts_2['avg_wordlength'].isna()].head()

,enter,ask,patent,Caphtorim,rule,hill,Genesis,authority,doth,confess,...,Ur,hundredth,answer,glad,Testament,suspicious,custom,text_sentence,text_source,avg_wordlength


In [927]:
Y = word_counts_2['text_source']

X = np.array(word_counts_2.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=0.4,random_state=0)

In [928]:
lr2 = LogisticRegression(penalty='l2')
lr2.fit(X_train,y_train)
print('Training set score:', lr2.score(X_train, y_train))
print('\nTest set score:', lr2.score(X_test, y_test))

Training set score: 0.9898477157360406

Test set score: 0.9277566539923955


/home/uzi/.venv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [929]:
pd.crosstab(y_test, lr2.predict(X_test), rownames=['Actual v'], colnames=['Predicted >'])

Predicted >,Austen,kjv
Actual v,,
Austen,115,4
kjv,15,129


In [931]:
print(cross_val_score(lr, X, Y, cv=5))

/home/uzi/.venv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/uzi/.venv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/uzi/.venv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/uzi/.venv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


[0.90909091 0.88636364 0.95454545 0.8778626  0.96153846]


/home/uzi/.venv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


> High accuracy scores across all folds and in the confusion matrix (and original test case).